---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Hypothesis Testing Impact of Recessions University Town Housing Prices

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. 

I will run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

Data:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

In [ ]:
#dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [ ]:
def get_list_of_university_towns():
    uni_towns = pd.read_csv ('university_towns.txt', sep=";", names=['Region Name'])
    uni_towns.insert(0, 'State', uni_towns['Region Name'].str.extract('(.*)\[edit\]', expand=False).ffill())
    uni_towns['Region Name'] = uni_towns['Region Name'].str.replace(r' \(.+$', '')
    uni_towns = uni_towns[~uni_towns['Region Name'].str.contains('\[edit\]')].reset_index(drop=True)
    uni_towns.rename(columns={'Region Name': \
                        'RegionName'}, inplace=True)
    return uni_towns

get_list_of_university_towns() 

In [ ]:
def get_recession_start():
    #Returns the year and quarter of the recession start time as a string value in a format such as 2005q3
    
    GDP = pd.read_excel ('gdplev.xls', header=4)
    GDP = GDP.drop(GDP.columns[[0, 1, 2, 3, 5, 7]], axis=1)
    GDP.rename(columns={'GDP in billions of chained 2009 dollars.1': \
                        'GDP', 'Unnamed: 4': 'year+quarter'}, inplace=True)
    GDP = GDP[GDP['year+quarter']>'2000'] 
    GDP = GDP.drop([GDP.index[0] , GDP.index[1]])
    GDP = GDP.iloc[0:]
    GDP.reset_index(inplace=True)
    GDP.drop(GDP.columns[0], axis=1, inplace=True)

    rec_start = GDP[(GDP['GDP'] > GDP['GDP'].shift(-1))&
                  (GDP['GDP'].shift(-1) > GDP['GDP'].shift(-2))]
    
    return rec_start.iloc[0,0]

get_recession_start()

In [ ]:
def get_recession_end():
    GDP = pd.read_excel ('gdplev.xls', header=4)
    GDP = GDP.drop(GDP.columns[[0, 1, 2, 3, 5, 7]], axis=1)
    GDP.rename(columns={'GDP in billions of chained 2009 dollars.1': \
                        'GDP', 'Unnamed: 4': 'year+quarter'}, inplace=True)
    GDP = GDP[GDP['year+quarter']>'2000'] 
    GDP = GDP.drop([GDP.index[0] , GDP.index[1]])
    GDP = GDP.iloc[0:]
    GDP.reset_index(inplace=True)
    GDP.drop(GDP.columns[0], axis=1, inplace=True)
    
    rec_start = get_recession_start()
    GDP = GDP[GDP['year+quarter'] >= rec_start]

    rec_end = GDP[(GDP['GDP'] > GDP['GDP'].shift(1))& 
                  (GDP['GDP'].shift(1) > GDP['GDP'].shift(2))]
               
    return rec_end.iloc[0,0]

get_recession_end()

In [ ]:
def get_recession_bottom():
    #Returns the year and quarter of the recession bottom time as a string value in a format such as 2005q3
    GDP = pd.read_excel ('gdplev.xls', header=4)
    GDP = GDP.drop(GDP.columns[[0, 1, 2, 3, 5, 7]], axis=1)
    GDP.rename(columns={'GDP in billions of chained 2009 dollars.1': \
                        'GDP', 'Unnamed: 4': 'year+quarter'}, inplace=True)
    GDP = GDP[GDP['year+quarter']>'2000'] 
    GDP = GDP.drop([GDP.index[0] , GDP.index[1]])
    GDP = GDP.iloc[0:]
    GDP.reset_index(inplace=True)
    GDP.drop(GDP.columns[0], axis=1, inplace=True)
    GDP = GDP[(GDP['year+quarter'] >= '2008q2') & (GDP['year+quarter'] <= '2009q4')]
    GDP.set_index(['year+quarter'], inplace=True)
    GDP = GDP[['GDP']].idxmin()
    
    return GDP.iloc[0]

get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    month_to_qtr = {'01':'q1','02':'q1','03':'q1','04':'q2','05':'q2','06':'q2',
                '07':'q3','08':'q3','09':'q3','10':'q4','11':'q4','12':'q4'}
    housing = pd.read_csv ('City_Zhvi_AllHomes.csv')
    housing = pd.DataFrame(housing.drop(['Metro','CountyName','SizeRank','RegionID'], axis=1))
    housing['State'].replace(states, inplace= True)
    housing = housing.set_index(['State','RegionName'])
    housing = housing.iloc[:,45:250]
    
    def quarters(col):
        if col.endswith(("01", "02", "03")):
            s = col[:4] + "q1"
        elif col.endswith(("04", "05", "06")):
            s = col[:4] + "q2"
        elif col.endswith(("07", "08", "09")):
            s = col[:4] + "q3"
        else:
            s = col[:4] + "q4"
        return s  
    housing = housing.groupby(quarters, axis = 1).mean()
    housing = housing.sort_index()
    
    return housing

convert_housing_data_to_quarters()

In [ ]:
def run_ttest():   
    #building the data that shows change in housing prices between start and bottom of recession
    u_towns = get_list_of_university_towns()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    houses = convert_housing_data_to_quarters()
    
    houses = houses.reset_index()
    houses['recession_diff'] = houses[rec_start] - houses[rec_bottom]
    
    u_towns_houses = pd.merge(houses, u_towns, how='inner', on=['State', 'RegionName'])
    u_towns_houses['ctown'] = True
    houses = pd.merge(houses, u_towns_houses, how='outer', on = ['State', 'RegionName',
                                                              rec_bottom, rec_start, 
                                                              'recession_diff'])
    houses['ctown'] = houses['ctown'].fillna(False)
    unitowns = houses[houses['ctown'] == True]
    not_unitowns = houses[houses['ctown'] == False]
    
    #comparing university town data to non-university town data
    t, p = ttest_ind(unitowns['recession_diff'].dropna(), not_unitowns['recession_diff'].dropna())
    different = True if p < 0.01 else False
    betters = "university town" if unitowns['recession_diff'].mean() < not_unitowns['recession_diff'].mean() else "non-university town"
    
    #different = True if t-test is True at p<0.01, different = False if otherwise. 
    #p is the value returned by scipy.stats.ttest_ind()
    #betters is university towns or non-university towns, depending on which has a lower mean price ratio. 
    return different,p,betters

run_ttest()